In [ ]:
!pip install sweetviz SUOD pyod prince

In [ ]:
import pandas as pd
import seaborn as sns
import sweetviz as svz
import prince
from sklearn.decomposition import PCA


In [ ]:
df = pd.read_csv("/kaggle/input/unsw-nb15-processed/UNSW_NB15_traintest_backdoor.csv")

In [ ]:
df

In [ ]:
# svz_result = svz.analyze(df,pairwise_analysis='off')

In [ ]:
# svz_result.show_html()

# MCA for categorical variable

In [ ]:
mca=prince.MCA(n_components=10,n_iter=20)
mca

In [ ]:
mca_results = mca.fit_transform(df.select_dtypes("int64").astype("string"))

In [ ]:
mca.scree_plot()

# PCA for numerical

In [ ]:
pca = PCA(2)
pca_results = pca.fit_transform(df.select_dtypes("float"))

In [ ]:
pca.explained_variance_ratio_

In [ ]:
rdf = pd.concat([pd.DataFrame(pca_results),pd.DataFrame(mca_results),df["class"]],axis=1)
rdf.columns = ["PC"+str(i) for i in range(12)]+["class"]

In [ ]:
# sns.pairplot(rdf)
svz.analyze(rdf,pairwise_analysis='off').show_html()

In [ ]:
from sklearn.metrics import balanced_accuracy_score,f1_score,precision_score,recall_score
def met(true,pred):
    print({
        "precision" : precision_score(true,pred),
        "recall_score" : recall_score(true,pred),
        "f1_score" : f1_score(true,pred),
        "balanced_accuracy_score" : balanced_accuracy_score(true,pred)
    })


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(rdf.drop(columns=["class"]),df["class"],stratify=df["class"])

In [ ]:
y_train.mean()

# Unsupervised ( with knowledge about average anomaly rate )

In [ ]:
from pyod.models.iforest import IForest
isf = IForest(500,contamination=0.0244)
isf.fit(x_train)
pred_train = isf.predict(x_train)
pred_test = isf.predict(x_test)
met(y_train,pred_train)
met(y_test,pred_test)


In [ ]:
from pyod.models.ecod import ECOD

ecod = ECOD(contamination=0.0244)
ecod.fit(x_train)
pred_train = ecod.predict(x_train)
pred_test = ecod.predict(x_test)
met(y_train,pred_train)
met(y_test,pred_test)

In [ ]:
from pyod.models.inne import INNE

inne = INNE(contamination=0.0244)
inne.fit(x_train)
pred_train = inne.predict(x_train)
pred_test = inne.predict(x_test)
met(y_train,pred_train)
met(y_test,pred_test)

In [ ]:
from pyod.models.mcd import MCD

mcd = MCD(contamination=0.0244,support_fraction =0.999)
mcd.fit(x_train)
pred_train = mcd.predict(x_train)
pred_test = mcd.predict(x_test)
met(y_train,pred_train)
met(y_test,pred_test)

In [ ]:
from pyod.models.loda import LODA

loda = LODA(contamination=0.0244)
loda.fit(x_train)
pred_train = loda.predict(x_train)
pred_test = loda.predict(x_test)
met(y_train,pred_train)
met(y_test,pred_test)

# Supervised

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rnd = RandomForestClassifier()
rnd.fit(x_train,y_train)
pred_train = rnd.predict(x_train)
pred_test = rnd.predict(x_test)
met(y_train,pred_train)
met(y_test,pred_test)


In [ ]:
from pyod.models.xgbod import XGBOD

xgbod = XGBOD([ECOD()])
xgbod.fit(x_train,y_train)
pred_train = xgbod.predict(x_train)
pred_test = xgbod.predict(x_test)
met(y_train,pred_train)
met(y_test,pred_test)

In [ ]:
#CONCLUSIONS :
# ECOD , INNE , MCD , IFOREST , XGBOD , supervised algorithms
# USE THE PYOD library and prince for the MCD